## MNIST Training Example

In [1]:
import sys
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import wandb
sys.argv = ['mnist']

### Training Settings 

In [2]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64)
parser.add_argument('--test-batch-size', type=int, default=1000)
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--momentum', type=float, default=0.5)
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1)
parser.add_argument('--log-interval', type=int, default=10)
args = parser.parse_args()

### Choosing the Device

In [3]:
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")

### Model Definition


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model = Net().to(device)

### Setup Data Loading

In [5]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
transform = transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('../data', train=False, transform=transform)
               
train_loader = torch.utils.data.DataLoader(train_data, batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size, shuffle=True, **kwargs)

### Optimizer

In [6]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

### Training Loop

In [7]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### Testing

In [8]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    example_images = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            # Save the first inbput tensor in each test batch as an example image
            example_images.append(wandb.Image(data[0], caption="Pred: {} Truth: {}".format(pred[0].item(), target[0])))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



### Main Loop


In [9]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.365850
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.305295
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.301407
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.316538
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.255810
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.224511
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.216569
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.181396
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.116898
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.045937
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.973613
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.969129
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.787625
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.735311
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.680114
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.485797
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.550642
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.196031
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.651938
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.979893
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.447727
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.339375
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.318713
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.544333
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.442704
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.250938
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.173111
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.245719
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.428325
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.473421
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.386385
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.282773
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.386286
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.489492
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.355502
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.276507
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.254574
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.604399
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.148974
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.380006


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.190724
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.174038
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.305022
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.379332
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.246032
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.275815
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.235348
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.311521
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.247506
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.201439
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.143781
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.315285
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.174074
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.299448
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.135620
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.362331
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.444928
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.291280
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.146700
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.302585


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.133659
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.457548
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.193121
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.138239
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.278107
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.228877
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.234770
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.180152
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.153760
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.203161
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.235536
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.263599
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.323912
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.236157
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.271685
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.243096
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.196616
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.160841
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.315276
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.176006


Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.295047

Test set: Average loss: 0.0650, Accuracy: 9808/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.305384
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.112376
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.143376
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.169786
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.130288
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.123633
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.065665
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.297876
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.126222
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.226079
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.191546
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.074306
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.254187
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.024953
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.206556
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.311448
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.121726
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.222843
Train Epoch: 8 [11

Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.175563
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.201009
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.198678
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.397387
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.140443
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.105467
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.093004
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.110417
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.201794
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.095915
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.057679
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.075863
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.079164
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.174503
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.100634
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.085447
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.115041
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.134821
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.174406
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.212015
